In [1]:
import os,sys
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.externals import joblib
import matplotlib.pyplot as plt
sys.path.append('../LIB/')
from env import ENV
from sklearn.preprocessing import normalize
from tqdm import tqdm

import os
from keras.layers import Dense,Input,LSTM,Bidirectional,Activation,Conv1D,GRU,CuDNNGRU,Flatten,BatchNormalization
from keras.callbacks import Callback
from keras.layers import Dropout,Embedding,GlobalMaxPooling1D, MaxPooling1D, Add, Flatten
from keras.preprocessing import text, sequence
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.models import Model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
import pickle
from sklearn.preprocessing.data import QuantileTransformer
from sklearn.utils import shuffle
import gc

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def scan_nan_portion(df):
    portions = []
    columns = []
    for col in df.columns:
        columns.append(col)
        portions.append(np.sum(df[col].isnull())/len(df))
    return pd.Series(data=portions, index=columns)



In [4]:
X_Train = pd.read_pickle(ENV.application_train_cleaned.value)
print('Train shape: {}'.format(X_Train.shape))

X_Test = pd.read_pickle(ENV.application_test_cleaned.value)
print('Test shape: {}'.format(X_Test.shape))

X_pre = pd.read_pickle(ENV.previous_application_cleaned_onehot.value)
print('Installment shape: {}'.format(X_pre.shape))


X_pre_combine = pd.read_pickle(ENV.previous_app_combine_rnnALL.value)
print('Installment shape: {}'.format(X_pre_combine.shape))


Train shape: (307511, 122)
Test shape: (48744, 121)
Installment shape: (1670214, 192)
Installment shape: (1670214, 322)


In [5]:
def get_embeddings_index(sorted_df,nor_ebd):
    embeddings_index={}
    words_values = sorted_df['words'].values
    for index in range(len(words_values)):
        embeddings_index  [words_values[index]] = nor_ebd[index,:]
    return embeddings_index

def create_document(sorted_df):
    #Create document
    ids = sorted_df.SK_ID_CURR.values
    words = sorted_df.words.values
    document_dicts = {}

    id_list = []
    document_list = []

    for index in range(len(ids)) :
        if document_dicts.get(ids[index]) is None:
            document_dicts[ids[index]] = []
        document_dicts[ids[index]].append(words[index])

    for key in document_dicts :
        document_dicts[key] = ' '.join(document_dicts[key])
        id_list.append(key)
        document_list.append(document_dicts[key])


    df_doc = pd.DataFrame({'SK_ID_CURR':id_list, 'text':document_list})  
    df_doc_mapping  = df_doc.set_index('SK_ID_CURR').text

    train = X_Train[['SK_ID_CURR','TARGET']].copy()
    test = X_Test[['SK_ID_CURR']].copy()
    train['text'] = train.SK_ID_CURR.map(df_doc_mapping).fillna('notfound')
    test['text'] = test.SK_ID_CURR.map(df_doc_mapping).fillna('notfound')
    return train,test


def get_train_ebdMat(train,test,embeddings_index):
    X_train = train["text"].str.lower()
    X_test = test["text"].str.lower()
    y_train = train["TARGET"].values
    tok=text.Tokenizer(num_words=max_features,lower=True,filters='')
    tok.fit_on_texts(list(X_train)+list(X_test))
    X_train=tok.texts_to_sequences(X_train)
    X_test=tok.texts_to_sequences(X_test)
    x_train=sequence.pad_sequences(X_train,maxlen=maxlen)
    x_test=sequence.pad_sequences(X_test,maxlen=maxlen)
    print('...get ebd mat')
    word_index = tok.word_index
    #prepare embedding matrix
    num_words = min(max_features, len(word_index) + 1)
    print('num of words: {}'.format(num_words))
    embedding_matrix = np.zeros((num_words, embed_size))
    print(embedding_matrix.shape)
    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return x_train,x_test,y_train,embedding_matrix,num_words

class_ratio =  sum(X_Train.TARGET ==0)/sum(X_Train.TARGET ==1)
class_ratio =  1
def get_rnn_model(num_words,embed_size,embedding_matrix):
    sequence_input = Input(shape=(maxlen, ))
    
    x = Embedding(num_words, embed_size, weights=[embedding_matrix],trainable = False)(sequence_input)
    x = BatchNormalization()(x)
    x = SpatialDropout1D(0.2)(x)
#     x = Bidirectional(GRU(16, return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
    x = Bidirectional(CuDNNGRU(8, return_sequences=True))(x)
    x = Conv1D(32, kernel_size = 3, padding = "valid", kernel_initializer = "glorot_uniform")(x)
    x = BatchNormalization()(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool]) 
#     x = BatchNormalization()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(1, activation="sigmoid")(x)
    model = Model(sequence_input, preds)
#     Adam,RMSprop,Adagrad,Adadelta,Adamax,Nadam
#     model.compile(loss='binary_crossentropy',optimizer=Adam(lr=1e-4),metrics=['accuracy'])
    model.compile(loss='binary_crossentropy',optimizer=Adam(),metrics=['accuracy'])
    return model

def train_each_epoch(x,y,batch_size,model):
    x,y = shuffle(x,y)
    model.fit(x, y, 
              batch_size=batch_size, 
              epochs=1,
              verbose=1,
              class_weight={0:1,1:class_ratio})
    return model

def load_model(model,filepath):
    model.load_weights(filepath)
    return model

def save_model(model, filepath):
    model.save_weights(filepath)

def train_each_fold(x,y,x_val,y_val,model,filepath,reportpath,predspath,
                    batch_size=512,total_epoch=40,patience=5,saving=True):
    ROC_AUC_SCORE = []
    for epoch in range(total_epoch):  
        model = train_each_epoch(x,y,batch_size,model)
        y_pred = model.predict(x_val,batch_size=3000,verbose=1)
        score = roc_auc_score(y_val,y_pred)
        if len(ROC_AUC_SCORE) == 0:
            if saving:
                save_model(model,filepath)
            best_score = 0 
            if saving:
                print('saving preds...')
                pickle.dump(y_pred,open(predspath,'wb'))
        else:
            best_score = max(ROC_AUC_SCORE)
            if score >= best_score:
                if saving:
                    print('saving model to... {}'.format(filepath))
                    save_model(model,filepath)
                    print('saving preds...')
                    pickle.dump(y_pred,open(predspath,'wb'))
        ROC_AUC_SCORE.append(score)
        if saving:
            print('saving report to... {}'.format(reportpath))
            pickle.dump(ROC_AUC_SCORE,open(reportpath,'wb'))
        print('======= current {} / {}'.format(epoch,total_epoch))
        print('previous best roc is {}'.format(best_score))
        print('current roc is {}'.format(score))
        try:
            best_round = ROC_AUC_SCORE.index(best_score)
        except ValueError:
            best_round = -1
        if len(ROC_AUC_SCORE) > patience + best_round:
            print('reach patience! end loop')
            break
            
def train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=512,total_epoch=400,patience=30):
    train_fold_index = pickle.load(open(ENV.train_fold_index.value,'rb'))
    val_fold_index = pickle.load(open(ENV.val_fold_index.value,'rb'))

    for fold in range(0,len(train_fold_index)):
        print('!!!!!!!! Begin fold: {}'.format(fold))
        train_index = train_fold_index[fold]
        val_index = val_fold_index[fold]
        X_tra = x_train[train_index]
        y_tra = y_train[train_index]
        X_val = x_train[val_index]
        y_val = y_train[val_index]
        print('preparing train/val done!')
        print('before evaluating: {}'.format(model_file))
        model_file_evl = model_file.format(fold)
        report_file_evl = report_file.format(fold)
        pred_file_evl = pred_file.format(fold)
        pred_test_file_evl = pred_test_file.format(fold)
        model = get_rnn_model(num_words,embed_size,embedding_matrix)
        train_each_fold(X_tra, y_tra, X_val, y_val,
                        model,
                        filepath=model_file_evl,reportpath=report_file_evl,predspath=pred_file_evl,
                        batch_size=batch_size,total_epoch=total_epoch,patience=patience)
        gc.collect()
        #### predict test
        model = load_model(model,model_file_evl)
        test_preds = model.predict(x_test,batch_size=1500,verbose=1)
        pickle.dump(test_preds,open(pred_test_file_evl,'wb'))
        print('\n')


# Previous_APPLICATION V1

In [6]:
qt = QuantileTransformer(n_quantiles=10000,output_distribution='normal')
trans_col = ['AMT_ANNUITY',
             'AMT_APPLICATION',
             'AMT_CREDIT',
             'AMT_DOWN_PAYMENT',
             'AMT_GOODS_PRICE',
             'RATE_DOWN_PAYMENT',
             'RATE_INTEREST_PRIMARY',
             'RATE_INTEREST_PRIVILEGED',
             'SELLERPLACE_AREA',
             'CNT_PAYMENT',
             'DAYS_FIRST_DRAWING',
             'DAYS_FIRST_DUE',
             'DAYS_LAST_DUE_1ST_VERSION',
             'DAYS_LAST_DUE',
             'DAYS_TERMINATION']
for col in trans_col:
    print(col)
    X_pre[col] = qt.fit_transform(X_pre[col].values.reshape(-1,1))

AMT_ANNUITY
AMT_APPLICATION
AMT_CREDIT
AMT_DOWN_PAYMENT
AMT_GOODS_PRICE
RATE_DOWN_PAYMENT
RATE_INTEREST_PRIMARY
RATE_INTEREST_PRIVILEGED
SELLERPLACE_AREA
CNT_PAYMENT
DAYS_FIRST_DRAWING
DAYS_FIRST_DUE
DAYS_LAST_DUE_1ST_VERSION
DAYS_LAST_DUE
DAYS_TERMINATION


In [7]:
label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
max_features = 1680000
# max 73
maxlen = 12
# 35
# embed_size = 35

sorted_df = X_pre.sort_values(['SK_ID_CURR','DAYS_DECISION']).copy()
col = 'DAYS_DECISION'
sorted_df[col] = qt.fit_transform(sorted_df[col].values.reshape(-1,1))
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('words')
embed_size = len(feature)
print('ebd size is {}'.format(embed_size))
###################################################
# feature.remove('DAYS_FIRST_DRAWING')
# feature.remove('DAYS_FIRST_DUE')
# feature.remove('DAYS_LAST_DUE_1ST_VERSION')
# feature.remove('DAYS_LAST_DUE')
# feature.remove('DAYS_TERMINATION')
###################################################
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
# model_file = ENV.previous_application_rnn.value
# report_file = ENV.previous_application_report.value
# pred_file = ENV.previous_application_preds.value
# pred_test_file = ENV.previous_application_preds_test.value
# train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=800,patience=45)

ebd size is 190
start normalize
get embedding
create document
get embedding Mat
...get ebd mat
num of words: 1670216
(1670216, 190)


In [8]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()

count    307511.000000
mean          4.650744
std           4.126887
min           1.000000
25%           2.000000
50%           3.000000
75%           6.000000
max          73.000000
Name: text, dtype: float64

In [8]:
aaaa

NameError: name 'aaaa' is not defined

# previous_APPLICATION V2

In [5]:
ins_cols = ['count_VERSION', 'count_INSTALLMENT', 'DAY_INS_SPAN',
       'DAY_ENTRY_SPAN', 'CNT_LATE_PAYMENT', 'CNT_LESS_PAYMENT',
       'TOTAL_AMT_INSTALMENT', 'TOTAL_AMT_PAYMENT', 'INSTAL_START_DAY',
       'FLAG_TOTAL_PAYMENT_LESS_THAN_INSTALMENT', 'OWE_PORTION']
pos_cols = [ 'MONTHS_BALANCE_MAX', 'MONTHS_BALANCE_MIN',
       'MONTHS_BALANCE_SPAN', 'CNT_INSTALMENT_MAX', 'CNT_INSTALMENT_MIN',
       'CNT_INSTALMENT_SPAN', 'SK_DPD_MAX', 'SK_DPD_MIN', 'SK_DPD_MEAN',
       'SK_DPD_COUNT', 'SK_DPD_SUM', 'SK_DPD_DEF_MAX', 'SK_DPD_DEF_MIN',
       'SK_DPD_DEF_MEAN', 'SK_DPD_DEF_COUNT', 'SK_DPD_DEF_SUM',
       'POSContractStatus_Active', 'POSContractStatus_Completed',
       'POSContractStatus_Signed', 'POSContractStatus_Approved',
       'POSContractStatus_Returned_to_the_store', 'POSContractStatus_Demand',
       'POSContractStatus_Canceled', 'POSContractStatus_XNA',
       'POSContractStatus_Amortized_debt', 
       'POS_LAST_STATUS_Completed', 'POS_LAST_STATUS_Active',
       'POS_LAST_STATUS_Amortized_debt',
       'POS_LAST_STATUS_Returned_to_the_store', 'POS_LAST_STATUS_Signed',
       'POS_LAST_STATUS_Demand', 'POS_LAST_STATUS_Approved',
       'POS_LAST_STATUS_Canceled', 'POS_FIRST_STATUS_Active',
       'POS_FIRST_STATUS_Approved', 'POS_FIRST_STATUS_Signed',
       'POS_FIRST_STATUS_Returned_to_the_store', 'POS_FIRST_STATUS_Completed',
       'POS_FIRST_STATUS_Canceled', 'POS_FIRST_STATUS_XNA']

cc_cols = ['CNT_DRAWINGS_CURRENT_MAX',
       'CNT_DRAWINGS_CURRENT_MIN', 'CNT_DRAWINGS_CURRENT_MEAN',
       'AMT_TOTAL_RECEIVABLE_MAX', 'AMT_TOTAL_RECEIVABLE_MIN',
       'AMT_TOTAL_RECEIVABLE_MEAN', 'AMT_RECIVABLE_MAX', 'AMT_RECIVABLE_MIN',
       'AMT_RECIVABLE_MEAN', 'AMT_RECEIVABLE_PRINCIPAL_MAX',
       'AMT_RECEIVABLE_PRINCIPAL_MIN', 'AMT_RECEIVABLE_PRINCIPAL_MEAN',
       'CC_SK_DPD_MAX', 'CC_SK_DPD_MIN', 'CC_SK_DPD_MEAN',
       'AMT_PAYMENT_TOTAL_CURRENT_MAX', 'AMT_PAYMENT_TOTAL_CURRENT_MIN',
       'AMT_PAYMENT_TOTAL_CURRENT_MEAN', 'AMT_DRAWINGS_CURRENT_MAX',
       'AMT_DRAWINGS_CURRENT_MIN', 'AMT_DRAWINGS_CURRENT_MEAN',
       'AMT_CREDIT_LIMIT_ACTUAL_MAX', 'AMT_CREDIT_LIMIT_ACTUAL_MIN',
       'AMT_CREDIT_LIMIT_ACTUAL_MEAN', 'AMT_BALANCE_MAX', 'AMT_BALANCE_MIN',
       'AMT_BALANCE_MEAN', 'CC_MONTHS_BALANCE_MAX', 'CC_MONTHS_BALANCE_MIN',
       'MONTHS_BALANCE_MEAN', 'CC_SK_DPD_DEF_MAX', 'CC_SK_DPD_DEF_MIN',
       'CC_SK_DPD_DEF_MEAN', 'AMT_INST_MIN_REGULARITY_MAX',
       'AMT_INST_MIN_REGULARITY_MIN', 'AMT_INST_MIN_REGULARITY_MEAN',
       'CNT_INSTALMENT_MATURE_CUM_MAX', 'CNT_INSTALMENT_MATURE_CUM_MIN',
       'CNT_INSTALMENT_MATURE_CUM_MEAN', 'AMT_DRAWINGS_POS_CURRENT_MAX',
       'AMT_DRAWINGS_POS_CURRENT_MIN', 'AMT_DRAWINGS_POS_CURRENT_MEAN',
       'AMT_DRAWINGS_ATM_CURRENT_MAX', 'AMT_DRAWINGS_ATM_CURRENT_MIN',
       'AMT_DRAWINGS_ATM_CURRENT_MEAN', 'CNT_DRAWINGS_ATM_CURRENT_MAX',
       'CNT_DRAWINGS_ATM_CURRENT_MIN', 'CNT_DRAWINGS_ATM_CURRENT_MEAN',
       'CNT_DRAWINGS_OTHER_CURRENT_MAX', 'CNT_DRAWINGS_OTHER_CURRENT_MIN',
       'CNT_DRAWINGS_OTHER_CURRENT_MEAN', 'CNT_DRAWINGS_POS_CURRENT_MAX',
       'CNT_DRAWINGS_POS_CURRENT_MIN', 'CNT_DRAWINGS_POS_CURRENT_MEAN',
       'AMT_DRAWINGS_OTHER_CURRENT_MAX', 'AMT_DRAWINGS_OTHER_CURRENT_MIN',
       'AMT_DRAWINGS_OTHER_CURRENT_MEAN', 'AMT_PAYMENT_CURRENT_MAX',
       'AMT_PAYMENT_CURRENT_MIN', 'AMT_PAYMENT_CURRENT_MEAN', 'Records_CNT',
       'NAME_CONTRACT_STATUS_Active', 'NAME_CONTRACT_STATUS_Completed',
       'NAME_CONTRACT_STATUS_Demand', 'NAME_CONTRACT_STATUS_Signed',
       'NAME_CONTRACT_STATUS_Sent_proposal', 'NAME_CONTRACT_STATUS_Refused',
       'NAME_CONTRACT_STATUS_Approved', 'NameContractFirstStatus_Active',
       'NameContractFirstStatus_Signed',
       'NameContractFirstStatus_Sent_proposal',
       'NameContractFirstStatus_Refused', 'NameContractFirstStatus_Demand',
       'NameContractFirstStatus_Approved', 'NameContractFirstStatus_Completed',
       'NameContractLastStatus_Active', 'NameContractLastStatus_Completed',
       'NameContractLastStatus_Demand', 'NameContractLastStatus_Signed']

In [6]:
qt = QuantileTransformer(n_quantiles=10000,output_distribution='normal')
trans_col_pre  = ['AMT_ANNUITY',
             'AMT_APPLICATION',
             'AMT_CREDIT',
             'AMT_DOWN_PAYMENT',
             'AMT_GOODS_PRICE',
             'RATE_DOWN_PAYMENT',
             'RATE_INTEREST_PRIMARY',
             'RATE_INTEREST_PRIVILEGED',
             'SELLERPLACE_AREA',
             'CNT_PAYMENT',
             'DAYS_FIRST_DRAWING',
             'DAYS_FIRST_DUE',
             'DAYS_LAST_DUE_1ST_VERSION',
             'DAYS_LAST_DUE',
             'DAYS_TERMINATION']

trans_col_pos = ['MONTHS_BALANCE_MAX',
             'MONTHS_BALANCE_MIN',
             'MONTHS_BALANCE_SPAN',
             'CNT_INSTALMENT_MAX',
             'CNT_INSTALMENT_MIN',
             'CNT_INSTALMENT_SPAN',
             'SK_DPD_MAX',
             'SK_DPD_MIN',
             'SK_DPD_MEAN',
             'SK_DPD_COUNT',
             'SK_DPD_SUM',
             'SK_DPD_DEF_MAX',
             'SK_DPD_DEF_MIN',
             'SK_DPD_DEF_MEAN',
             'SK_DPD_DEF_COUNT',
             'SK_DPD_DEF_SUM']

trans_col_ins = ['count_VERSION',
             'count_INSTALLMENT',
             'DAY_INS_SPAN',
             'DAY_ENTRY_SPAN',
             'CNT_LATE_PAYMENT',
             'CNT_LESS_PAYMENT',
             'TOTAL_AMT_INSTALMENT',
             'TOTAL_AMT_PAYMENT',
             'INSTAL_START_DAY',
             'OWE_PORTION']

trans_col_cc = [ 'CNT_DRAWINGS_CURRENT_MAX',
       'CNT_DRAWINGS_CURRENT_MIN', 'CNT_DRAWINGS_CURRENT_MEAN',
       'AMT_TOTAL_RECEIVABLE_MAX', 'AMT_TOTAL_RECEIVABLE_MIN',
       'AMT_TOTAL_RECEIVABLE_MEAN', 'AMT_RECIVABLE_MAX', 'AMT_RECIVABLE_MIN',
       'AMT_RECIVABLE_MEAN', 'AMT_RECEIVABLE_PRINCIPAL_MAX',
       'AMT_RECEIVABLE_PRINCIPAL_MIN', 'AMT_RECEIVABLE_PRINCIPAL_MEAN',
       'CC_SK_DPD_MAX', 'CC_SK_DPD_MIN', 'CC_SK_DPD_MEAN',
       'AMT_PAYMENT_TOTAL_CURRENT_MAX', 'AMT_PAYMENT_TOTAL_CURRENT_MIN',
       'AMT_PAYMENT_TOTAL_CURRENT_MEAN', 'AMT_DRAWINGS_CURRENT_MAX',
       'AMT_DRAWINGS_CURRENT_MIN', 'AMT_DRAWINGS_CURRENT_MEAN',
       'AMT_CREDIT_LIMIT_ACTUAL_MAX', 'AMT_CREDIT_LIMIT_ACTUAL_MIN',
       'AMT_CREDIT_LIMIT_ACTUAL_MEAN', 'AMT_BALANCE_MAX', 'AMT_BALANCE_MIN',
       'AMT_BALANCE_MEAN', 'CC_MONTHS_BALANCE_MAX', 'CC_MONTHS_BALANCE_MIN',
       'MONTHS_BALANCE_MEAN', 'CC_SK_DPD_DEF_MAX', 'CC_SK_DPD_DEF_MIN',
       'CC_SK_DPD_DEF_MEAN', 'AMT_INST_MIN_REGULARITY_MAX',
       'AMT_INST_MIN_REGULARITY_MIN', 'AMT_INST_MIN_REGULARITY_MEAN',
       'CNT_INSTALMENT_MATURE_CUM_MAX', 'CNT_INSTALMENT_MATURE_CUM_MIN',
       'CNT_INSTALMENT_MATURE_CUM_MEAN', 'AMT_DRAWINGS_POS_CURRENT_MAX',
       'AMT_DRAWINGS_POS_CURRENT_MIN', 'AMT_DRAWINGS_POS_CURRENT_MEAN',
       'AMT_DRAWINGS_ATM_CURRENT_MAX', 'AMT_DRAWINGS_ATM_CURRENT_MIN',
       'AMT_DRAWINGS_ATM_CURRENT_MEAN', 'CNT_DRAWINGS_ATM_CURRENT_MAX',
       'CNT_DRAWINGS_ATM_CURRENT_MIN', 'CNT_DRAWINGS_ATM_CURRENT_MEAN',
       'CNT_DRAWINGS_OTHER_CURRENT_MAX', 'CNT_DRAWINGS_OTHER_CURRENT_MIN',
       'CNT_DRAWINGS_OTHER_CURRENT_MEAN', 'CNT_DRAWINGS_POS_CURRENT_MAX',
       'CNT_DRAWINGS_POS_CURRENT_MIN', 'CNT_DRAWINGS_POS_CURRENT_MEAN',
       'AMT_DRAWINGS_OTHER_CURRENT_MAX', 'AMT_DRAWINGS_OTHER_CURRENT_MIN',
       'AMT_DRAWINGS_OTHER_CURRENT_MEAN', 'AMT_PAYMENT_CURRENT_MAX',
       'AMT_PAYMENT_CURRENT_MIN', 'AMT_PAYMENT_CURRENT_MEAN', 'Records_CNT',
       'NAME_CONTRACT_STATUS_Active', 'NAME_CONTRACT_STATUS_Completed',
       'NAME_CONTRACT_STATUS_Demand', 'NAME_CONTRACT_STATUS_Signed',
       'NAME_CONTRACT_STATUS_Sent_proposal', 'NAME_CONTRACT_STATUS_Refused',
       'NAME_CONTRACT_STATUS_Approved']
for col in trans_col_pre + trans_col_pos +trans_col_ins +trans_col_cc:
    print(col)
    X_pre_combine[col] = qt.fit_transform(X_pre_combine[col].values.reshape(-1,1))

AMT_ANNUITY
AMT_APPLICATION
AMT_CREDIT
AMT_DOWN_PAYMENT
AMT_GOODS_PRICE
RATE_DOWN_PAYMENT
RATE_INTEREST_PRIMARY
RATE_INTEREST_PRIVILEGED
SELLERPLACE_AREA
CNT_PAYMENT
DAYS_FIRST_DRAWING
DAYS_FIRST_DUE
DAYS_LAST_DUE_1ST_VERSION
DAYS_LAST_DUE
DAYS_TERMINATION
MONTHS_BALANCE_MAX
MONTHS_BALANCE_MIN
MONTHS_BALANCE_SPAN
CNT_INSTALMENT_MAX
CNT_INSTALMENT_MIN
CNT_INSTALMENT_SPAN
SK_DPD_MAX
SK_DPD_MIN
SK_DPD_MEAN
SK_DPD_COUNT
SK_DPD_SUM
SK_DPD_DEF_MAX
SK_DPD_DEF_MIN
SK_DPD_DEF_MEAN
SK_DPD_DEF_COUNT
SK_DPD_DEF_SUM
count_VERSION
count_INSTALLMENT
DAY_INS_SPAN
DAY_ENTRY_SPAN
CNT_LATE_PAYMENT
CNT_LESS_PAYMENT
TOTAL_AMT_INSTALMENT
TOTAL_AMT_PAYMENT
INSTAL_START_DAY
OWE_PORTION
CNT_DRAWINGS_CURRENT_MAX
CNT_DRAWINGS_CURRENT_MIN
CNT_DRAWINGS_CURRENT_MEAN
AMT_TOTAL_RECEIVABLE_MAX
AMT_TOTAL_RECEIVABLE_MIN
AMT_TOTAL_RECEIVABLE_MEAN
AMT_RECIVABLE_MAX
AMT_RECIVABLE_MIN
AMT_RECIVABLE_MEAN
AMT_RECEIVABLE_PRINCIPAL_MAX
AMT_RECEIVABLE_PRINCIPAL_MIN
AMT_RECEIVABLE_PRINCIPAL_MEAN
CC_SK_DPD_MAX
CC_SK_DPD_MIN
CC_SK_

In [7]:
# cc_retain = ['NameContractFirstStatus_Active',
#        'NameContractFirstStatus_Signed',
#        'NameContractFirstStatus_Sent_proposal',
#        'NameContractFirstStatus_Refused', 'NameContractFirstStatus_Demand',
#        'NameContractFirstStatus_Approved', 'NameContractFirstStatus_Completed',
#        'NameContractLastStatus_Active', 'NameContractLastStatus_Completed',
#        'NameContractLastStatus_Demand', 'NameContractLastStatus_Signed']

# for each in cc_retain:
#     cc_cols.remove(each)
# X = X_pre_combine.drop(cc_cols,axis=1)
X = X_pre_combine.copy()

In [8]:

print('start')
label_mapping = X_Train.set_index('SK_ID_CURR').TARGET
test_mapping = pd.Series(index=X_Test.SK_ID_CURR, data=1)

#previous application
#10001358
max_features = 1680000
# max 73
maxlen = 12

sorted_df = X.sort_values(['SK_ID_CURR','DAYS_DECISION'])
sorted_df['words'] = sorted_df.index.astype(str)
feature = list(sorted_df.columns)
feature.remove('SK_ID_PREV')
feature.remove('SK_ID_CURR')
feature.remove('words')
ebd = sorted_df[feature].values
#normalize
print('start normalize')
# nor_ebd = normalize(ebd, norm='max',axis=0)
nor_ebd = ebd
print('get embedding')
embed_size = len(feature)
print('ebd size is {}'.format(embed_size))
embeddings_index = get_embeddings_index(sorted_df,nor_ebd)
print('create document')
train,test = create_document(sorted_df)
print('get embedding Mat')
x_train,x_test,y_train,embedding_matrix,num_words = get_train_ebdMat(train,test,embeddings_index)
model_file = ENV.previous_application_rnn.value
report_file = ENV.previous_application_report.value
pred_file = ENV.previous_application_preds.value
pred_test_file = ENV.previous_application_preds_test.value
train_5_folds(model_file,report_file,pred_file,pred_test_file,batch_size=2000,total_epoch=500,patience=35)

start
start normalize
get embedding
ebd size is 320
create document
get embedding Mat
...get ebd mat
num of words: 1670216
(1670216, 320)
!!!!!!!! Begin fold: 3
preparing train/val done!
before evaluating: ../LIB/../../data/rnn/previous_application/fold_{}.hdf5
Epoch 1/1
61502/61502 [==============================] - 0s 4us/step
saving preds...
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 0 / 500
previous best roc is 0
current roc is 0.6005970785687338
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/previous_application/fold_3.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 1 / 500
previous best roc is 0.6005970785687338
current roc is 0.6265531268096285
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/previous_application/fold_3.hdf5
saving pre

saving preds...
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 20 / 500
previous best roc is 0.6848737547531855
current roc is 0.6860679297950248
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 21 / 500
previous best roc is 0.6860679297950248
current roc is 0.685447595237874
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 22 / 500
previous best roc is 0.6860679297950248
current roc is 0.6847156698935895
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 23 / 500
previous best roc is 0.6860679297950248
current roc is 0.6854198792704633
Epoch 1/1
61502/61502 [==============================] - 0

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 44 / 500
previous best roc is 0.6876519313508431
current roc is 0.6827808876776224
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 45 / 500
previous best roc is 0.6876519313508431
current roc is 0.6811974447382015
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 46 / 500
previous best roc is 0.6876519313508431
current roc is 0.6842774120541291
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_3.pkl
======= current 47 / 500
previous best roc is 0.6876519313508431
current roc is 0.6835050443150209
Epoc

saving preds...
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= current 1 / 500
previous best roc is 0.6064891266351049
current roc is 0.6533408143209797
Epoch 1/1
61502/61502 [==============================] - 0s 3us/step
saving model to... ../LIB/../../data/rnn/previous_application/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= current 2 / 500
previous best roc is 0.6533408143209797
current roc is 0.6609304984402127
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/previous_application/fold_4.hdf5
saving preds...
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= current 3 / 500
previous best roc is 0.6609304984402127
current roc is 0.6701759748052447
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving model to... ../LIB/../../data/rnn/previous_applicatio

Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= current 47 / 500
previous best roc is 0.685579560032031
current roc is 0.6800198748519072
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= current 48 / 500
previous best roc is 0.685579560032031
current roc is 0.6812947231542026
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= current 49 / 500
previous best roc is 0.685579560032031
current roc is 0.6780310841607602
Epoch 1/1
61502/61502 [==============================] - 0s 2us/step
saving report to... ../LIB/../../data/rnn/previous_application/report_fold_4.pkl
======= current 50 / 500
previous best roc is 0.685579560032031
current roc is 0.6805663099619969
Epoch 1/

In [9]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()

count    307511.000000
mean          4.650744
std           4.126887
min           1.000000
25%           2.000000
50%           3.000000
75%           6.000000
max          73.000000
Name: text, dtype: float64

In [ ]:
a = train.text.apply(lambda x: len(x.split(' ')))
a.describe()